# Learn IFC
IFC is a standard format to store 3D CAD data. In Python, *ifcopenshell* is a package to access IFC data.

This notebook demonstrates how to use the package to fetch CAD data, shows use cases, and proves assumptions/concepts.


**Outline**
- [Load IFC File](#learn-ifc)
- [IFC Data Structure](#ifc-structure)
    - [Entity Relations](#ifcrelaggregates)
    - [Object Placement](#ifclocalplacement)
    - [Axis and Placement](#ifcaxis2placement3d)
    - [Shape Definition](#ifcproductdefinitionshape)
    - [Geometry Representation](#ifcshaperepresentation)
    - [Boundary Representation](#ifcfacetedbrep)
    - [Face Bounding](#ifcface)
    - [Bounds on Face](#ifcfaceouterbound)
    - [Cartesian Point](#ifccartesianpoint)
- [Use Cases](#use-cases)
    - [Geometry Transformation](#study-geometry-transformation)
    - [Smallest Bounding Box](#prove-smallest-bounding-box)
- [PoC](#prove-of-assumptionsconcepts)
    - [Common Axises?](#how-many-axises-are-common-in-one-assembly)


In [ ]:
import numpy as np
import ifcopenshell
import pandas as pd

## Load IFC file

In [ ]:
filePath = "ifc-data/sample.ifc"
ifcModel = ifcopenshell.open(filePath)
ifcModel

## IFC Structure
Each text line in IFC is an entity

There are many types of entity e.g. IfcElementAssembly, IfcRelAggregates and IfcBeam etc.

Each type of entity has different attributes.

Each entity relates to other entities where you can see reference entities IDs in attributes.

In [ ]:
# generic attributes
entityID = 139817
elementAssembly = ifcModel.by_id(entityID)
assert elementAssembly.is_entity(), f"{entityID} is not an entity"
assert elementAssembly.is_a() == "IfcElementAssembly", f"{entityID} is not an element assembly"
print(elementAssembly.id(), elementAssembly.Name, elementAssembly.Tag)
print(elementAssembly.get_info())
print(elementAssembly.to_string())

### IfcRelAggregates
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifckernel/lexical/ifcrelaggregates.htm</a>

Aggregation relationship (mapping) entity shows related and relating entities.

Can interprete that a related entity has many relating entities.

*Relating Entity* ---|---(RelatingObject)---> **RelAggresgates Entity** ---(RelatedObjects)---> *Related Entity*<br>
*Relating Entity* ---|<br>
*Relating Entity* ---|<br>

For example,

Part ---|--- Assembly<br>
Part ---|<br>
Part ---|<br>


In [ ]:
relations = elementAssembly.IsDecomposedBy[0]
print(relations)
print(relations.RelatingObject) # refer to elementAssembly entity
parts = relations.RelatedObjects # referred by many entities
print(parts[0])

### IfcLocalPlacement
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcgeometricconstraintresource/lexical/ifclocalplacement.htm</a>

A mapping entity shows details about where this element is in 3D

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<i>Object</i>&emsp;<i>Object</i>&emsp;<i>Object</i><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;-------------------<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Objects on this placement (PlacesObject)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
<i>IfcLocalPlacement</i> ---|---Referred by (ReferencedByPlacements)---> <b>IfcLocalPlacement</b> ---Refer to (PlacementRelTo)---> <i>IfcLocalPlacement</i><br>
<i>IfcLocalPlacement</i> ---|&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; |<br>
<i>IfcLocalPlacement</i> ---|&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(RelativePlacement)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<i>IfcAxis2Placement3D</i><br>

In [ ]:
placement = elementAssembly.ObjectPlacement
print(placement)
print(placement.PlacesObject) # objects/entities (elementAssembly) that are on this placement 
print(placement.PlacementRelTo) # Reference to Object that provides the relative placement by its local coordinate system
print(placement.ReferencedByPlacements) # referred by LocalPlacement entities

### IfcAxis2Placement3D
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x/FINAL/HTML/ifcgeometryresource/lexical/ifcaxis2placement3d.html</a>

Info of local axises againt global axises

<img src="https://standards.buildingsmart.org/IFC/RELEASE/IFC2x/FINAL/HTML/ifcgeometryresource/lexical/figures/IfcAxis2Placement3D-Layout1.gif" style="background-color: white">

In [ ]:
print(placement.RelativePlacement) # Geometric placement that defines the transformation from the related coordinate system into the relating
print(placement.RelativePlacement.Axis)
print(placement.RelativePlacement.Location)
print(placement.RelativePlacement.RefDirection)

### IfcProductDefinitionShape
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcrepresentationresource/lexical/ifcproductdefinitionshape.htm</a>

Product/part/object shape definitions

In [ ]:
partDef = parts[0].Representation
print(partDef)
print(partDef.Representations) # IfcShapeRepresentation

### IfcShapeRepresentation
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcrepresentationresource/lexical/ifcshaperepresentation.htm</a>

Geometric representation of a product

In [ ]:
partGeom = partDef.Representations[0]
print(partGeom)
print(partGeom.ContextOfItems)
print(partGeom.RepresentationType)
print(partGeom.RepresentationIdentifier)
print(partGeom.OfProductRepresentation)
# print(partGeom.LayerAssignments)
print(partGeom.Items) # IfcFacetedBrep

### IfcFacetedBrep
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcgeometricmodelresource/lexical/ifcfacetedbrep.htm</a>

Boundary representation model in which all faces are planar and all edges are straight lines

In [ ]:
facet = partGeom.Items[0]
print(facet)
print(facet.Outer) # IfcFace

### IfcFace
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifctopologyresource/lexical/ifcface.htm</a>

Contain face bounds

In [ ]:
faces = facet.Outer[0]
print(faces)
print(faces[0].Bounds)

### IfcFaceOuterBound
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifctopologyresource/lexical/ifcfaceouterbound.htm</a>

Define an outer boundary on the face

In [ ]:
bound = faces[0].Bounds[0]
print(bound)
print(bound.Bound) # IfcCartesianPoint
print(bound.Orientation)

### IfcCartesianPoint
<a>https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcgeometryresource/lexical/ifccartesianpoint.htm</a>

Three dimensional rectangular Cartesian coordinate system

In [ ]:
points = bound.Bound[0]
print(points) # list of points
print(points[0].Coordinates) # point coordinates 
# Coordinates[1] is the X coordinate, Coordinates[2] is the Y coordinate, and Coordinates[3] is the Z coordinate

## Use Cases

### Study Geometry Transformation

In [ ]:
print(parts[0])
print(parts[0].ObjectPlacement)
print(parts[0].ObjectPlacement.RelativePlacement)

In [ ]:
locOnGlob = parts[0].ObjectPlacement.RelativePlacement.Location[0]
locOnGlob = np.array(locOnGlob)
print("Local point on global", locOnGlob)

translateMat = np.identity(4)
translateMat[:3, 3] = locOnGlob
print("Translation matrix\n", translateMat)

ZDirOnGlob = parts[0].ObjectPlacement.RelativePlacement.Axis[0]
ZDirOnGlob = np.array(ZDirOnGlob)
print("Local Z axis unit vector on global", ZDirOnGlob)

XDirOnGlob = parts[0].ObjectPlacement.RelativePlacement.RefDirection[0]
XDirOnGlob = np.array(XDirOnGlob)
print("Local X axis unit vector on global", XDirOnGlob)

YDirOnGlob = np.cross(ZDirOnGlob, XDirOnGlob)
print("Local Y axis unit vector on global", YDirOnGlob)

AllDirOnGlob = np.array([ZDirOnGlob, XDirOnGlob, YDirOnGlob])
AllDirOnGlob = AllDirOnGlob.T
print("All local axises on global\n", AllDirOnGlob)

rotateMat = np.identity(4)
rotateMat[:3, :3] = AllDirOnGlob
print("Rotation matrix\n", rotateMat)

In [ ]:
localPoint = np.array([[10000], [1000], [10000], [1]])
print("Dummy local point\n", localPoint)
print("Vector magnitude", np.sqrt(np.sum(np.power(localPoint, 2))))
print("Translated\n", translateMat @ localPoint)
print("Rotated\n", rotateMat @ localPoint)
print("Vector magnitude", np.sqrt(np.sum(np.power(rotateMat @ localPoint, 2))))

### Prove Smallest Bounding Box

In [ ]:
elemID = 50852
assembly = ifcModel.by_id(elemID)
parts = assembly.IsDecomposedBy[0].RelatedObjects
print("ASSEMBLY", assembly)

minPartName = None
minBBox = None
minDim = None
minVol = float("inf")

for refPart in parts:
    # print("REF", refPart)
    
    loc = refPart.ObjectPlacement.RelativePlacement.Location[0]
    ax = refPart.ObjectPlacement.RelativePlacement.Axis[0]
    refDir = refPart.ObjectPlacement.RelativePlacement.RefDirection[0]
    
    # Translation matrix
    translateMat = np.identity(4)
    translateMat[:3, 3] = loc
    
    # Rotation matrix. Axis and RefDirection are normalized ???
    rotateMat = np.identity(4)
    # rotateMat[:3, :3] = np.array([ax, refDir, np.cross(ax, refDir)]).T # ChatGPT incorrect ???
    rotateMat[:3, :3] = np.array([ax, refDir, np.cross(ax, refDir)]).T
    
    # Combine into onee transformation matrix
    refTransformatMatrix = translateMat @ rotateMat
    # print("REF TRANSFORMATION\n", refTransformatMatrix)

    minX = float("inf")
    minY = float("inf")
    minZ = float("inf")
    maxX = float("-inf")
    maxY = float("-inf")
    maxZ = float("-inf")

    for part in parts:
        # print("PART", part, end=" ")    
        try:
            faces = part.Representation.Representations[0].Items[0].Outer[0]
            # print("")
        except:
            # print("NO FACE")
            continue

        loc = part.ObjectPlacement.RelativePlacement.Location[0]
        ax = part.ObjectPlacement.RelativePlacement.Axis[0]
        refDir = part.ObjectPlacement.RelativePlacement.RefDirection[0]
        # Translation matrix
        translateMat = np.identity(4)
        translateMat[:3, 3] = loc
        
        # Rotation matrix. Axis and RefDirection are normalized ???
        rotateMat = np.identity(4)
        # rotateMat[:3, :3] = np.array([ax, refDir, np.cross(ax, refDir)]).T # chatGPT incorrect ???
        rotateMat[:3, :3] = np.array([refDir, np.cross(ax, refDir), ax]).T
        
        # Combine translation and rotation to form the transformation matrix
        transformMat = translateMat @ rotateMat
        # print("TRANSFORMATION\n", transformMat)
        
        transformMat = np.linalg.inv(refTransformatMatrix) @ transformMat
        # print("TRANSFORMATION TO REF\n", transformMat)
        
        for face in faces:
            # Get vertices from each face 
            bounding = face.Bounds[0].Bound[0]

            # Loop over vertices
            for point in bounding:
                point = point[0]
                point = np.array(point + (1,))

                pointOnRef = transformMat @ point
                
                minX = min(minX, pointOnRef[0])
                minY = min(minY, pointOnRef[1])
                minZ = min(minZ, pointOnRef[2])
                maxX = max(maxX, pointOnRef[0])
                maxY = max(maxY, pointOnRef[1])
                maxZ = max(maxZ, pointOnRef[2])
                
    # bouding box of whole assembly on reference frame
    partName = str(refPart)
    bbox = np.array([[minX, minY, minZ], [maxX, maxY, maxZ]])
    dim = bbox[1] - bbox[0]
    vol = dim[0] * dim[1] * dim[2]
    # print(f"REF: {partName}\n   Dim: {dim}\n   vol: {vol}\n   bbox:\n{bbox}")
    
    # find min vol
    if vol < minVol:
        minPartName = partName
        minBBox = bbox
        minDim = dim
        minVol = vol
    
print(f"*** Min bbox:\n   Ref to {minPartName}\n   Dim: {minDim}\n   vol: {minVol}\n   bbox:\n{minBBox}")
    

## Prove of Assumptions/Concepts

### How many axises are common in one assembly?
*= Each object has its own axises*

In [ ]:
axisesDF = pd.DataFrame([obj.ObjectPlacement.RelativePlacement.id() for obj in elementAssembly.IsDecomposedBy[0].RelatedObjects], columns=['id'])
axisesDF.iloc[list(axisesDF['id'].value_counts() > 1)]